In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle

from pprint import pprint

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [ ]:
today = datetime.now().strftime("%Y%m%d")
# print(today)

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()
    
# print(api_key)

lista_puestos = pd.read_excel("Lista_Puestos.xlsx", header = None)

lista_puestos.columns = ["jobs"]

In [ ]:
lista_puestos

In [ ]:
with open("lista_puestos_ingles.txt", "r") as file:
    lista_puestos_ingles = file.read()
    
lista_puestos_ingles = lista_puestos_ingles.split("\n")

lista_puestos_ingles

In [ ]:
[x for x in [x.replace("Developer", "") for x in lista_puestos_ingles] if len(x.split()) > 1]

In [ ]:
latam = [["com.ar", "Argentina"]         ,
         ["com.bo", "Bolivia"]           ,
         ["com.br", "Brazil"]            ,
         ["ca", "Canada"]            ,
         ["cl", "Chile"]             ,
         ["com.co", "Colombia"]          ,
         ["co.cr", "Costa Rica"]        ,
         ["com.cu", "Cuba"]              ,
         ["com.do", "Dominican Republic"],
         ["com.ec", "Ecuador"]           ,
         ["com.sv", "El Salvador"]       ,
         ["com.gt", "Guatemala"]         ,
#          ["gy", "Guyana"]            ,
#          ["ht", "Haiti"]             ,
         ["hn", "Honduras"]          ,
#          ["jm", "Jamaica"]           ,
         ["com.mx", "Mexico"]            ,
         ["com.ni", "Nicaragua"]         ,
         ["com.pa", "Panama"]            ,
         ["com.py", "Paraguay"]          ,
         ["com.pe", "Peru"]              ,
         ["com.pr", "Puerto Rico"]       ,
         ["com.uy", "Uruguay"]           ,
#          ["ve", "Venezuela"]
        ]

latam

In [ ]:
%%time

df = pd.DataFrame()

for abr, country in latam[::-1]:
    
    for q in lista_puestos_ingles:

        try:
            for pagination in range(100):
                
#                 sleep(0.3)

                print(f"{q:60}{pagination}")

                q_params = {"q"             : q,
                            "api_key"       : api_key,
                            "location"      : country.title(),
                            "start"         : pagination*10,
#                             "gl"            : abr,
#                             "hl"            : "es",
#                             "google_domain" : f"google.{abr}",
#                             "chips"         : "date_posted:year"
                            }

                response = serpapi.job_search(**q_params)


                if ("error" in response) or (response.get("jobs_results") == None) or (len(response.get("jobs_results")) < 10):
                    break
                
                now = datetime.now()
                
                with open(file = f"json_searches_latam/{str(now.timestamp()).replace('.', '')}.pkl", mode = "bw") as file:
                    pickle.dump(response, file)

                df_response = pd.json_normalize(response["jobs_results"])

                df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

                df_response["country_search"] = country.title()
            #     df_response = df_response[["job_id", "title", "company_name", "location", "source", "description", "country", "date_posted", "contract_type", "remote_work"]]

                df = pd.concat(objs = [df, df_response], ignore_index = True)
        except:
            print(f"Error {q} ***************************************************************")
            
        
            
            
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True)

df.to_csv(f"extraccion_{today}_latam2.csv", index = False)
df.to_pickle(f"extraccion_{today}_latam2.pkl")

In [ ]:
response

In [ ]:
df_new = df.copy()
df_new.drop_duplicates(subset = "description", inplace = True)
df_new.reset_index(drop = True, inplace = True)

df_new.shape

In [ ]:
df_new["date_posted"] = datetime.strptime("2023-08-02", "%Y-%m-%d").date()
df_new

In [ ]:
%%time

job_types = ["Full-time", "Part-time", "Contractor", "Internship"]

# Drops Duplicates
df_new = df_new.drop_duplicates("job_id").reset_index(drop = True) 

# Cleans "via"
df_new["via"] = df_new["via"].apply(lambda x : clean.clean_source(x) if not pd.isna(x) else x)

# Cleans "location"
df_new["location"] = df_new["location"].apply(lambda x : clean.clean_location(x))

# Cleans "contract_type"
# df_new["schedule_type"] = df_new["schedule_type"].apply(lambda x : clean.clean_contract_type(x) if x not in job_types else x)

# Cleans "created_date"
df_new["posted_at"] = df_new["posted_at"].apply(lambda x : clean.transform_date(x))

df_new["date_posted"] = [clean.get_date(x, y) for x, y in df_new[["date_posted", "posted_at"]].values]

# Creates "tech_skills"
df_new["tech_skills"] = df_new["description"].apply(lambda x : clean.get_skills(x) if not pd.isna(x) else x)

In [ ]:
df_new.to_csv("latam 004.csv", index = False)